In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

stock_name = "RELIANCE.NS"
stock_ = yf.Ticker(stock_name)

# get all stock info
all_info = stock_.info

# List of keys to filter
info_keys = ['symbol', 'shortName', 'industry', 'sector', 'longBusinessSummary','financialCurrency','marketCap', 'previousClose',
             'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'volume', 'averageVolume10days',
             'dividendRate', 'dividendYield', 'exDividendDate', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield',
             'payoutRatio', 'fiveYearAvgDividendYield','beta', 'trailingPE', 'forwardPE',
             'enterpriseValue', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'twoHundredDayAverage',
             'profitMargins',  'bookValue', 'priceToBook', 'trailingEps', 'forwardEps','enterpriseToRevenue', 'enterpriseToEbitda',
             'firstTradeDateEpochUtc', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 'totalCashPerShare',
             'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity',
             'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow',
             'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins','trailingPegRatio', 'netIncomeToCommon'
             ]

In [3]:
# Filter the dictionary
filtered_all_info = {key: all_info[key] for key in info_keys if key in all_info}

In [5]:
# filtered_all_info

In [6]:
#converting to DataFrame
data_all_info = pd.DataFrame([filtered_all_info])

In [8]:
# function to get 1Y and 5Y Growth Strategy
def find_growth(attribute:str):
  data = stock_.income_stmt
  _1Y_growth = (data.iloc[:,0].loc[attribute] - data.iloc[:,1].loc[attribute])/data.iloc[:,1].loc[attribute]
  _5Y_growth = (data.iloc[:,0].loc[attribute] - data.iloc[:,3].loc[attribute])/data.iloc[:,3].loc[attribute]
  _5Y_growth = _5Y_growth/3
  return _1Y_growth, _5Y_growth

In [9]:

# Feature Engineering 
_1Y_EBITDA_Growth, _5Y_EBITDA_Growth = find_growth(attribute='EBITDA')
_1Y_TotalRevenue_Growth, _5Y_TotalRevenue_Growth = find_growth(attribute='Total Revenue')
_1Y_TotalIncome_Growth, _5Y_TotalIncome_Growth = find_growth(attribute='Net Income')
_1Y_PreTaxIncome_Growth, _5Y_PreTaxIncome_Growth = find_growth(attribute='Pretax Income')

In [10]:
# Adding Features
data_all_info['netIncomeMargin'] = data_all_info['netIncomeToCommon']/data_all_info['totalRevenue']
data_all_info['% Away From 52High'] = (data_all_info['fiftyTwoWeekHigh']/data_all_info['currentPrice'])-1
data_all_info['% Away From 52Low'] = (data_all_info['currentPrice']/data_all_info['fiftyTwoWeekLow'])-1

data_all_info['1Y Total Revenue Growth'] = _1Y_TotalRevenue_Growth
data_all_info['5Y Total Revenue Growth'] = _5Y_TotalRevenue_Growth

data_all_info['1Y EBITDA Growth'] = _1Y_EBITDA_Growth
data_all_info['5Y EBITDA Growth'] = _5Y_EBITDA_Growth


data_all_info['1Y PreTax Income Growth'] = _1Y_PreTaxIncome_Growth
data_all_info['5Y PreTax Income Growth'] = _5Y_PreTaxIncome_Growth

data_all_info['1Y Net Income Growth'] = _1Y_TotalIncome_Growth
data_all_info['5Y Net Income Growth'] = _5Y_TotalIncome_Growth

In [11]:
data_all_info

,symbol,shortName,industry,sector,longBusinessSummary,financialCurrency,marketCap,previousClose,currentPrice,targetHighPrice,...,% Away From 52High,% Away From 52Low,1Y Total Revenue Growth,5Y Total Revenue Growth,1Y EBITDA Growth,5Y EBITDA Growth,1Y PreTax Income Growth,5Y PreTax Income Growth,1Y Net Income Growth,5Y Net Income Growth
0,RELIANCE.NS,RELIANCE INDUSTRIES LTD,Oil & Gas Refining & Marketing,Energy,Reliance Industries Limited engages in hydroca...,INR,18108381134848,1343.9,1338.15,2830.0,...,0.202257,0.164596,0.026462,0.309929,0.156705,0.258125,0.113572,0.2961,0.043762,0.139045


In [13]:
stock_.cash_flow

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Free Cash Flow,59050000000.0,-259560000000.0,105090000000.0,-788790000000.0,NaN
Repayment Of Debt,-350550000000.0,-290590000000.0,-406470000000.0,-872400000000.0,NaN
Issuance Of Debt,696100000000.0,359360000000.0,593430000000.0,332110000000.0,NaN
Issuance Of Capital Stock,70000000.0,400000000.0,397670000000.0,132150000000.0,NaN
Capital Expenditure,-1528830000000.0,-1409880000000.0,-1001450000000.0,-1058370000000.0,NaN
End Cash Position,972250000000.0,686640000000.0,361780000000.0,173970000000.0,NaN
Other Cash Adjustment Outside Changein Cash,7200000000.0,-17660000000.0,9410000000.0,240000000.0,NaN
Beginning Cash Position,686640000000.0,361780000000.0,173970000000.0,309200000000.0,NaN
Changes In Cash,278410000000.0,342520000000.0,178400000000.0,-135470000000.0,NaN
Financing Cash Flow,-166460000000.0,104550000000.0,172890000000.0,1019040000000.0,NaN
